In [ ]:
import json
from collections import Counter
import pandas as pd
import os

In [ ]:
in_dir = "/mnt/nis_lab_research/data/coco_files/clean/far_shah-b1-b2_cln/result.json"
# out_dir = "./output"
out_dir = in_dir[0:-11]

In [ ]:
with open(in_dir) as f:
    coco_json_obj = json.load(f)


In [ ]:
ann_list = coco_json_obj["annotations"]
cat_list = coco_json_obj["categories"]
img_list = coco_json_obj["images"]

In [ ]:
# Getting overall coco breakdown

cat_occ_list = []
for ann in ann_list:
    cat_occ_list.append(ann["category_id"])
    
cat_keys = Counter(cat_occ_list).keys()
cat_vals = Counter(cat_occ_list).values()

cat_prcnt = []
vals_tot = sum(list(cat_vals))
for val in list(cat_vals):
    cat_prcnt.append(val/vals_tot)

cat_name_list = []
for key in cat_keys:
    for item in cat_list:
        if key == item["id"]:
            cat_name_list.append(item["name"])

df_tot = pd.DataFrame({"cat_key": list(cat_keys), "cat_name": cat_name_list, "cat_count": list(cat_vals), "cat_prcnt": cat_prcnt}).sort_values(by="cat_count", ascending=False)

df_tot
           

In [ ]:
# Exporting overall breakdown to CSV

df_tot.to_csv(os.path.join(out_dir, "coco_ann_tot.csv"))


In [ ]:
# Getting individual img analysis

f = open(os.path.join(out_dir, "coco_ann_full.txt"), "w+")
f.write("FULL COCO JSON FILE ANALYSIS\n\n")

img_id_list = []

for item in img_list: 
    img_id_list.append([item["id"], item["file_name"]])

for each in img_id_list:
    
    f.write("Image " + str(each[0]) + " > " + each[1] + "\n")
    single_img_list = []
    
    for item in ann_list:
        if each[0] == item["image_id"]:
            single_img_list.append(item["category_id"])
            cat_keys = Counter(single_img_list).keys()
            cat_vals = Counter(single_img_list).values()

            cat_name_list = []
            for key in cat_keys:
                for thing in cat_list:
                    if key == thing["id"]:
                        cat_name_list.append(thing["name"])
    
    for i in range(0, len(cat_keys)):
        f.write("   " + str(list(cat_keys)[i]) + " " + cat_name_list[i] + " > " + str(list(cat_vals)[i]) + "\n")
        
    f.write("\n")
       
f.close()
